In [ ]:
from pybit._v5_market import MarketHTTP
import time
from pprint import pprint
import matplotlib.pyplot as plt
import datetime
import timesfm

market = MarketHTTP(
    testnet=False
)

In [2]:
#--------------------------------------------------------------------------------------------------------

In [ ]:
tfm = timesfm.TimesFm(
    context_len=1024,
    horizon_len=100,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend='gpu',
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
intrvl = 5
data = []
last_time = time.time()*1000
for i in range(1):
    end_time = last_time
    start_time = end_time - (999*intrvl*60000)
    
    result = market.get_index_price_kline(
        category='linear',
        symbol='BTCUSDT',
        interval=str(intrvl),
        start=start_time,
        end=end_time,
        limit=1000
    )
    data.extend(result.get('result').get('list'))
    last_time = int(start_time) - intrvl*60000
data = data[::-1]
    
print(len(data))
# print(data)

# Извлекаем время и цены из данных
times = [int(item[0]) for item in data]
prices = [float(item[4]) for item in data]

In [ ]:
#==================================================================
shift = 33
#==================================================================
learn_prices = prices[:-shift] if shift else prices
start_price = learn_prices[-1]
learn_times = times[:-shift] if shift else times

[fdata], [experimental_quantile_forecast] = tfm.forecast(
    [learn_prices],
    # freq=[0 for _ in range(2)],
    # window_size=2
    freq=[0],
)
print(start_price)
# print(fdata)
# print(learn_prices[-1])
# print(prices[-(shift+1)])
# print('-----------------')
# print(times)
# print(experimental_quantile_forecast)

In [ ]:
times2 = [datetime.datetime.fromtimestamp(time / 1000) for time in times]

point = times[-shift] if shift else times[-1]

fcst_times = [t for t in range(point - 60000*intrvl, point + ((len(fdata)-1)*60000*intrvl), 60000*intrvl)]
fcst_times = [datetime.datetime.fromtimestamp(time / 1000) for time in fcst_times]

print(datetime.datetime.fromtimestamp(point/1000))

In [ ]:
plt.figure(figsize=(13, 8))
diffs = []
values = {}
for i in range(0, 10):
    diff = abs(experimental_quantile_forecast[0][i] - start_price)
    diffs.append(diff)
    values[i] = []
    for checkpoint in experimental_quantile_forecast:
        values[i].append(checkpoint[i])
    plt.plot(fcst_times, values[i], marker='', linestyle='dotted', linewidth=1, color='orange')
minindex = diffs.index(min(diffs))
min_values = values[minindex]

plt.plot(fcst_times, min_values, marker='', linestyle='-', color='red')
plt.plot(fcst_times, fdata, marker='', linestyle='--', color='green')
plt.plot(times2[-500:], prices[-500:], marker='', linestyle='-', color='black')
# plt.plot(times2, prices, marker='', linestyle='-', color='black')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(f'BTCUSDT - {intrvl}m')
plt.grid(True)
plt.show()